# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Подготовка данных

In [171]:
# Импорт библиотек
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator

RANDOM_SEED = 2022

# Инициализация Spark-сессии
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

In [172]:
# Чтение файла
df_housing = spark.read.load('/datasets/housing.csv', format="csv", sep=",", inferSchema=True, header="true")

# Вывод первых 5 строк
df_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [173]:
# вывед названий колонок 
pd.DataFrame(df_housing.dtypes, columns=['column', 'type']).head(10)

,column,type
0,longitude,double
1,latitude,double
2,housing_median_age,double
3,total_rooms,double
4,total_bedrooms,double
5,population,double
6,households,double
7,median_income,double
8,median_house_value,double
9,ocean_proximity,string


В колонках датасета содержатся следующие данные:
- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.

Столбец median_house_value является целевым признаком.

In [174]:
# вывод базовой статистики
df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [175]:
num_columns = [c for c in df_housing.columns if c != 'ocean_proximity']

quantile = df_housing.approxQuantile(num_columns, [0.5], 0)
print(quantile)

[[-118.49], [34.26], [29.0], [2127.0], [435.0], [1166.0], [409.0], [3.5347], [179700.0]]


In [176]:
df_quantile = pd.DataFrame(quantile, columns= ['quantile_50'])
df_quantile.index = num_columns

df_housing_describe = df_housing.select(*num_columns).describe().toPandas().set_index('summary').T.join(df_quantile)
columns = ['count', 'mean', 'stddev','min', 'quantile_50', 'max']

df_housing_describe[columns]

,count,mean,stddev,min,quantile_50,max
longitude,20640,-119.56970445736148,2.003531723502584,-124.35,-118.4900,-114.31
latitude,20640,35.6318614341087,2.135952397457101,32.54,34.2600,41.95
housing_median_age,20640,28.639486434108527,12.58555761211163,1.0,29.0000,52.0
total_rooms,20640,2635.7630813953488,2181.6152515827944,2.0,2127.0000,39320.0
total_bedrooms,20433,537.8705525375618,421.38507007403115,1.0,435.0000,6445.0
population,20640,1425.4767441860465,1132.46212176534,3.0,1166.0000,35682.0
households,20640,499.5396802325581,382.3297528316098,1.0,409.0000,6082.0
median_income,20640,3.8706710029070246,1.899821717945263,0.4999,3.5347,15.0001
median_house_value,20640,206855.81690891474,115395.61587441359,14999.0,179700.0000,500001.0


Изучив данные, можно сделать вывод, что в столбце total_bedrooms имеются пропуски, однако их количество не более 1 %. К тому же, количество спален напрямую зависит от количества комнат, спален примерно в 5 раз меньше, чем комнат. Однако мы оставим замену пропусков средним значением.

In [177]:
# Проверим корреляцию признаков
df_housing.corr('total_rooms', 'total_bedrooms')

0.9201961721166215

In [178]:
# Заполним пропуски средним значением
df_housing = df_housing.na.fill(df_housing.select(F.mean(df_housing['total_bedrooms'])).collect()[0][0])

In [179]:
df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375639,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.26659232552385,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


## Подготовка признаков

В датафрейме имеются данные, содержащие следующие признаки:
- текстовые признаки - ocean_proximity;
- числовые признаки - 'longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income';
- целевой признак.

Выделим эти признаки.

In [180]:
categorical_col = 'ocean_proximity'
numerical_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
target = 'median_house_value'

### Трансформация категорийных признаков

Трансформируем категорийные признаки с помощью трансформера StringIndexer, который переведет текстовую категорию в числовую.

In [181]:
indexer = StringIndexer(inputCol=categorical_col, outputCol=categorical_col+'_idx')
df_housing = indexer.fit(df_housing).transform(df_housing)

In [182]:
cols = [c for c in df_housing.columns if c.startswith(categorical_col)]
df_housing.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



Применим OHE-кодирование для категорий:

In [183]:
encoder = OneHotEncoder(inputCol='ocean_proximity_idx', outputCol=categorical_col+'_ohe')
df_housing = encoder.fit(df_housing).transform(df_housing)
cols = [c for c in df_housing.columns if c.startswith(categorical_col)]
df_housing.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



Объединим признаки в один вектор, с которым работает ML-алгоритм:

In [184]:
categorical_assembler = VectorAssembler(inputCols=['ocean_proximity_ohe'], outputCol='categorical_features')
df_housing = categorical_assembler.transform(df_housing)

In [185]:
pd.DataFrame(df_housing.take(3),
             columns=df_housing.columns)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_idx,ocean_proximity_ohe,categorical_features
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)"
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)"
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)"


### Трансформачия числовых признаков

Проведем шкалирование числовых значений, что бы сильные выбросы не смещали предсказание модели.

In [186]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol='numerical_features')
df_housing = numerical_assembler.transform(df_housing)

In [187]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled')
df_housing = standardScaler.fit(df_housing).transform(df_housing)

In [188]:
df_housing.dtypes

[('longitude', 'double'),
 ('latitude', 'double'),
 ('housing_median_age', 'double'),
 ('total_rooms', 'double'),
 ('total_bedrooms', 'double'),
 ('population', 'double'),
 ('households', 'double'),
 ('median_income', 'double'),
 ('median_house_value', 'double'),
 ('ocean_proximity', 'string'),
 ('ocean_proximity_idx', 'double'),
 ('ocean_proximity_ohe', 'vector'),
 ('categorical_features', 'vector'),
 ('numerical_features', 'vector'),
 ('numerical_features_scaled', 'vector')]

### Объединение категорийных и числовых признаков

Соберем трансформированные категорийные и числовые признаки с помощью VectorAssembler

In [189]:
all_features = ['categorical_features', 'numerical_features_scaled']
final_assembler = VectorAssembler(inputCols=all_features, outputCol='features')
df_housing = final_assembler.transform(df_housing)
df_housing.select('features').show(3)

+--------------------+
|            features|
+--------------------+
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
+--------------------+
only showing top 3 rows



### Разделение на выборки

Разделим выборку на 2 части - обучающую (80%) и тестовую (20%).

In [190]:
train_data, test_data = df_housing.randomSplit([.8, .2], seed = RANDOM_SEED)
print(train_data.count(), test_data.count())

16418 4222


Подготовили признаки для последующего обучения модели, трансформирорвали категорийные и числовые признаки с помощью трансформера StringIndexer, дополнительно провели OHE-кодирование, провели шкалирование значений StandardScaler. Далее собрали трансформированные категорийные и числовые признаки с помощью VectorAssembler. Разделили выборку на 2 части - обучающую (80%) и тестовую (20%).

## Обучение моделей

Построим две модели линейной регрессии на разных наборах данных:
- используя все данные из файла;
- используя только числовые переменные, исключив категориальные.

Для построения модели будем использовать оценщик LinearRegression из библиотеки MLlib, после чнго сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2.

Для обучения будем использовать вектор features - будем использовать все данные, numerical_features_scaled - только числовые переменные. Все полученные данные сохраним в список

In [191]:
# Создаем список, куда будем вносить результаты модели в зависимости от набора данных
data_table = [['features_name', 'RMSE', 'MAE', 'R2']]

# Создадим цикл для используемых признаков
for col in ['features', 'numerical_features_scaled']:
    
    # обучим модель на обучающей выборке
    lr = LinearRegression(featuresCol=col, labelCol=target)
    model = lr.fit(train_data)
    
    # предскажим на тестовой выборке
    predictions = model.transform(test_data)
    
    # выделим предсказания и ответы в отдельную переменную
    results = predictions.select(['prediction', target])

    rmse = RegressionEvaluator(labelCol=target, metricName='rmse').evaluate(predictions)
    mae = RegressionEvaluator(labelCol=target, metricName='mae').evaluate(predictions)
    r2 = RegressionEvaluator(labelCol=target, metricName='r2').evaluate(predictions)
    
    # добавим результаты в список для сравнения
    data_table.append([col, rmse, mae, r2])
print(pd.DataFrame(data_table[1:], columns=data_table[0]))  


22/06/14 18:44:53 WARN Instrumentation: [52965055] regParam is zero, which might cause numerical instability and overfitting.
22/06/14 18:44:54 WARN Instrumentation: [52965055] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/06/14 18:44:55 WARN Instrumentation: [f282911d] regParam is zero, which might cause numerical instability and overfitting.


               features_name          RMSE           MAE        R2
0                   features  68480.413421  49849.341084  0.653622
1  numerical_features_scaled  69207.629961  50848.475110  0.646226


## Анализ результатов

In [192]:
pd.DataFrame(data_table[1:], columns=data_table[0])

,features_name,RMSE,MAE,R2
0,features,68480.413421,49849.341084,0.653622
1,numerical_features_scaled,69207.629961,50848.475110,0.646226


Обучив модель линейной регрессии на двух выборках - используя все данные из файла и используя только числовые переменные, исключив категориальные, можно сделать вывод о том, что модель линейной регрессии на выборке со всеми файлами предсказывает точнее по метрикам RMSE, MAE и R2.

In [193]:
spark.stop() 